In [273]:
from __future__ import print_function
import copy
import pandas as pd
import numpy as np
import librosa
import seaborn as sb
import matplotlib.pyplot as plt
import itertools
import re
import random
import gc
import os
from os import listdir
from os.path import isfile, join
from numpy import median, diff
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization

In [274]:
steps_per_bar = 48
class SongFile:
    def __init__(self, key, folder, stepfile, music_file):
        misc = pd.read_csv('data/{0}_misc.csv'.format(key)).values
        self.beat_importance = pd.read_csv('generated_data/{0}_importance_generated.csv'.format(key), converters={'0': lambda x: float(x)}).values
        self.notes = pd.read_csv('data/{0}_notes.csv'.format(key), converters={'0': lambda x: str(x)}).values
        self.folder = folder
        self.name = key.split('~')[1]
        self.music_name = music_file
        self.stepfile_name = stepfile
        self.offset = misc[0][0]
        self.beat_length = 60. / misc[1][0]
        self.bpm = misc[1][0]
        self.extension = music_file.split('.')[1]

In [296]:
songs_to_use = pd.read_csv('data/songs_to_use.csv').values
save_files = listdir('data')
save_files_generated = listdir('generated_data')
songs = {}
for song_data in songs_to_use:
    key = song_data[0]
    if '{0}_misc.csv'.format(key) in save_files and '{0}_importance_generated.csv'.format(key) in save_files_generated:
        songs[key] = SongFile(key, song_data[1], song_data[2], song_data[3])

In [302]:
beats_to_track = 48
num_classes = 3
class_map = {
    '0': 0,
    '1': 1,
    '2': 1,
    '3': 0,
    '4': 1,
    'M': 2
}

def get_is_note(i, notes):
    if i < 0:
        return [0, 0, 0, 0]
    return [char == '1' for char in notes[i][0]]

def get_is_mine(i, notes):
    if i < 0:
        return [0, 0, 0, 0]
    return [char == 'M' for char in notes[i][0]]

def get_note_class(note):
    if i < 0:
        return 0
    return class_map[note] if note in class_map else 0

def get_row_classes(row):
    return [get_note_class(note) for note in row[0]]

X = []
y = []
for key in songs:
    beat_importance = songs[key].beat_importance
    notes = songs[key].notes
    length = min(len(beat_importance), len(notes))
    is_note = np.array([get_is_note(j, notes) for j in range(-beats_to_track, length)])
    is_mine = np.array([get_is_note(j, notes) for j in range(-beats_to_track, length)])
    importances = [0 if j < 0 else beat_importance[j][0] for j in range(-beats_to_track, length)]
    for i in range(length):
        if get_row_classes(notes[i]) != [0, 0, 0, 0] or random.randint(0, 5) == 0:
            X_row = np.concatenate((is_note[i:i + beats_to_track - 1].flatten(), is_mine[i:i + beats_to_track - 1].flatten(), importances[i:i + beats_to_track]), axis=0)
            X.append(X_row)
            y.append(get_row_classes(notes[i]))

# TODO: filter some 0's from y here to make output more fun


X = np.array(X)
y = np.array(y)

In [303]:
len(X)

169174

In [306]:
def build_model():
    model = Sequential()

    model.add(Dense(200, input_dim=424, init='uniform'))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))

    model.add(Dense(500, init='uniform'))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))

    model.add(Dense(500, init='uniform'))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes, init='uniform'))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                               optimizer='adadelta',
                               metrics=['accuracy'])
    
    return model

In [307]:
#TODO: build models for either side instead of one left one right
models = [build_model() for i in range(4)]

In [ ]:
for i in range(4):
    y_transformed = [row[i] for row in y]
    y_one_hot = np.zeros((len(y_transformed), num_classes))
    y_one_hot[np.arange(len(y_transformed)), y_transformed] = 1

    models[i].fit(np.array(X), np.array(y_one_hot), nb_epoch=2, batch_size=5)

Epoch 1/8
 16440/169174 [=>............................] - ETA: 101s - loss: 0.4821 - acc: 0.8779

In [256]:
outputs = {
    0: '0',
    1: '1',
    2: 'M'
}
# TODO: make 3/4's less common by penalizing here or figure out how in model
def get_output_for_note(note_class_predictions):
    return outputs[np.argmax(note_class_predictions)]

def get_output_for_index(y, i):
    columns = [y[j][i] for j in range(4)]
    return ''.join([get_output_for_note(note) for note in columns])

In [257]:
def get_output(song):
    X = []
    beat_importance = songs[key].beat_importance
    notes = songs[key].notes
    for i in range(len(notes)):
        is_note = np.array([get_is_note(j, notes) for j in range(i - beats_to_track, i - 1)]).flatten()
        is_mine = np.array([get_is_mine(j, notes) for j in range(i - beats_to_track, i - 1)]).flatten()
        importances = [0 if j < 0 else beat_importance[j][0] for j in range(i - beats_to_track, i)]
        X_row = np.concatenate((is_note, is_mine, importances), axis=0)
        X.append(X_row)
        
    X = np.array(X)
    y = [models[i].predict(X) for i in range(4)]
    output = [get_output_for_index(y, i) for i in range(len(notes))]
    return output

In [258]:
def write_song_header(output_stepfile, song):
    keys = ['TITLE', 'MUSIC', 'OFFSET', 'SAMPLESTART', 'SAMPLELENGTH', 'SELECTABLE', 'BPMS']
    header_info = {
        'TITLE': song.name,
        'MUSIC': '{0}.{1}'.format(song.name, song.extension),
        'OFFSET': -song.offset,
        'SAMPLESTART': song.offset + 32 * song.beat_length,
        'SAMPLELENGTH': 32 * song.beat_length,
        'SELECTABLE': 'YES',
        'BPMS': '0.000={:.3f}'.format(song.bpm)
    }
    
    for key in keys:
        print ("#{0}:{1};".format(key, str(header_info[key])), file=output_stepfile)
        
def write_step_header(output_stepfile, song):
    print("\n//---------------dance-single - J. Zukewich----------------", file=output_stepfile)
    print ("#NOTES:", file=output_stepfile)
    for detail in ['dance-single', 'J. Zukewich', 'Expert', '9', '0.242,0.312,0.204,0.000,0.000']:
        print ('\t{0}:'.format(detail), file=output_stepfile)
    
    for i in range(len(song.predicted_notes)):
        row = song.predicted_notes[i]
        print (row, file=output_stepfile)
        if i % steps_per_bar == steps_per_bar - 1:
            print (",", file=output_stepfile)

    print ("0000;", file=output_stepfile)
    
def step_song(song):
    if song.name + '.sm' in os.listdir(song.folder) and not song.name + '.sm.backup' in os.listdir(song.folder):
        os.rename(song.stepfile_name, song.stepfile_name + '.backup')
            
    output_stepfile=open(song.stepfile_name, 'w')
    write_song_header(output_stepfile, song)
    write_step_header(output_stepfile, song)
    output_stepfile.close()

In [259]:
for key in songs:
    song = songs[key]
    song.predicted_notes = get_output(song)
    step_song(song)

In [260]:
songs

{'In The Groove~Anubis': <__main__.SongFile at 0x12a17e438>,
 'In The Groove~Bend Your Mind': <__main__.SongFile at 0x13eef2668>,
 'In The Groove~Boogie Down': <__main__.SongFile at 0x13eef27f0>,
 'In The Groove~Bouff': <__main__.SongFile at 0x132866908>,
 'In The Groove~Bubble Dancer': <__main__.SongFile at 0x12704f9e8>}